# 🌍 Real Market Factor Mining Demo

## 真实市场因子挖掘验证

使用来自不同行业的代表性股票，在多个时间周期下验证因子挖掘框架的有效性。

### 股票池

| 代码 | 公司 | 行业 |
|------|------|------|
| AAPL | Apple | 科技 |
| MSFT | Microsoft | 科技 |
| JPM | JPMorgan Chase | 金融 |
| XOM | ExxonMobil | 能源 |
| JNJ | Johnson & Johnson | 医疗 |

### 测试周期
- **日频 (Daily)**: 短期交易信号
- **周频 (Weekly)**: 中期调仓信号
- **月频 (Monthly)**: 长期配置信号


In [ ]:
# 导入库
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# 导入框架
from src.backtest import ICAnalyzer, PortfolioBacktest
from src.beta_model import RiskFactorModel

print("✓ 库加载完成!")


## 1. 数据下载

从Yahoo Finance下载真实的股票数据。为了获得更有意义的因子测试，我们扩展股票池到30只不同行业的代表性股票。


In [ ]:
# 定义股票池
STOCKS = {
    # 科技
    'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Alphabet', 
    'META': 'Meta', 'NVDA': 'NVIDIA', 'AMZN': 'Amazon',
    # 金融
    'JPM': 'JPMorgan', 'BAC': 'BankOfAmerica', 'GS': 'GoldmanSachs', 
    'MS': 'MorganStanley', 'C': 'Citigroup',
    # 能源
    'XOM': 'ExxonMobil', 'CVX': 'Chevron', 'COP': 'ConocoPhillips',
    # 医疗
    'JNJ': 'Johnson&Johnson', 'UNH': 'UnitedHealth', 'PFE': 'Pfizer', 
    'MRK': 'Merck', 'ABBV': 'AbbVie',
    # 消费
    'WMT': 'Walmart', 'PG': 'P&G', 'KO': 'CocaCola', 
    'PEP': 'PepsiCo', 'COST': 'Costco',
    # 工业
    'CAT': 'Caterpillar', 'DE': 'Deere', 'HON': 'Honeywell', 
    'UPS': 'UPS', 'BA': 'Boeing'
}

symbols = list(STOCKS.keys())
print(f"股票池: {len(symbols)} 只股票")
print(f"行业分布: 科技(6), 金融(5), 能源(3), 医疗(5), 消费(5), 工业(5)")


In [ ]:
# 下载数据 (2020-2024)
print("📊 正在下载数据...")
data = yf.download(symbols, start='2020-01-01', end='2024-12-01', progress=True, auto_adjust=True)

# 提取数据
close = data['Close']
volume = data['Volume']
high = data['High']
low = data['Low']
returns = close.pct_change()

print(f"\n✓ 数据下载完成!")
print(f"  日期范围: {close.index[0].strftime('%Y-%m-%d')} 至 {close.index[-1].strftime('%Y-%m-%d')}")
print(f"  交易日数: {len(close)}")
print(f"  股票数量: {len(close.columns)}")


In [ ]:
# 可视化累计收益
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 所有股票累计收益
cum_returns = (1 + returns).cumprod()
ax1 = axes[0]
for col in cum_returns.columns:
    ax1.plot(cum_returns.index, cum_returns[col], alpha=0.6, linewidth=1, label=col)
ax1.set_title('所有股票累计收益曲线', fontsize=14)
ax1.set_xlabel('日期')
ax1.set_ylabel('累计收益')
ax1.axhline(1, color='black', linestyle='--', alpha=0.5)
ax1.legend(loc='upper left', ncol=3, fontsize=8)

# 行业平均表现
sectors = {
    'Tech': ['AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'AMZN'],
    'Finance': ['JPM', 'BAC', 'GS', 'MS', 'C'],
    'Energy': ['XOM', 'CVX', 'COP'],
    'Healthcare': ['JNJ', 'UNH', 'PFE', 'MRK', 'ABBV'],
    'Consumer': ['WMT', 'PG', 'KO', 'PEP', 'COST'],
    'Industrial': ['CAT', 'DE', 'HON', 'UPS', 'BA']
}

ax2 = axes[1]
colors = plt.cm.tab10(np.linspace(0, 1, len(sectors)))
for (sector, tickers), color in zip(sectors.items(), colors):
    valid_tickers = [t for t in tickers if t in cum_returns.columns]
    if valid_tickers:
        sector_avg = cum_returns[valid_tickers].mean(axis=1)
        ax2.plot(sector_avg.index, sector_avg, linewidth=2, label=sector, color=color)

ax2.set_title('各行业平均累计收益', fontsize=14)
ax2.set_xlabel('日期')
ax2.set_ylabel('累计收益')
ax2.axhline(1, color='black', linestyle='--', alpha=0.5)
ax2.legend(loc='upper left')

plt.tight_layout()
plt.show()


## 2. 因子构建

我们构建5个经典因子进行测试：

1. **动量因子 (Momentum)**: 过去N期收益率
2. **低波动率因子 (Low Volatility)**: 低波动率异象
3. **反转因子 (Reversal)**: 短期反转效应
4. **成交量因子 (Volume)**: 异常成交量
5. **价格区间因子 (Price Range)**: 基于高低价的波动度量


In [ ]:
# 因子构建函数
def build_factors(returns, volume, high, low, close, lookback=20):
    """构建所有因子"""
    factors = {}
    
    # 1. 动量因子
    factors['Momentum'] = returns.rolling(lookback).apply(
        lambda x: (1 + x).prod() - 1, raw=True
    ).shift(1)  # 跳过最近1期避免前视偏差
    
    # 2. 低波动率因子 (取负: 低波动=高因子值)
    factors['LowVolatility'] = -returns.rolling(lookback).std()
    
    # 3. 反转因子 (取负: 过去跌->预期涨)
    factors['Reversal'] = -returns.rolling(5).apply(
        lambda x: (1 + x).prod() - 1, raw=True
    )
    
    # 4. 成交量因子 (异常成交量)
    avg_volume = volume.rolling(lookback).mean()
    factors['Volume'] = (volume / avg_volume - 1)
    
    # 5. 价格区间因子 (取负: 低波动=高因子值)
    daily_range = (high - low) / close
    factors['PriceRange'] = -daily_range.rolling(lookback).mean()
    
    return factors

# 构建日频因子
factors_daily = build_factors(returns, volume, high, low, close, lookback=20)

print("✓ 日频因子构建完成!")
for name, factor in factors_daily.items():
    print(f"  {name}: {factor.dropna().shape}")


## 3. 因子IC分析 (日频)


In [ ]:
# 因子IC分析函数
def analyze_factor_ic(factor, returns, factor_name):
    """分析单个因子的IC"""
    # 转换为长格式
    factor_long = factor.stack()
    factor_long.name = 'factor'
    factor_long.index.names = ['date', 'stock']
    factor_long = factor_long.to_frame()
    
    returns_long = returns.stack()
    returns_long.name = 'return'
    returns_long.index.names = ['date', 'stock']
    returns_long = returns_long.to_frame()
    
    # IC分析
    ic_analyzer = ICAnalyzer(method='spearman', forward_periods=1, min_observations=5)
    ic_series = ic_analyzer.compute_ic(factor_long, returns_long, 
                                       factor_col='factor', return_col='return')
    ic_stats = ic_analyzer.compute_ic_stats()
    
    return ic_series, ic_stats

# 分析所有日频因子
print("📊 日频因子IC分析结果:")
print("="*70)
print(f"{'因子名称':15s} | {'IC Mean':>10s} | {'IC Std':>10s} | {'IR':>8s} | {'t-stat':>8s} | {'显著性':>6s}")
print("-"*70)

daily_ic_results = {}
for name, factor in factors_daily.items():
    ic_series, ic_stats = analyze_factor_ic(factor, returns, name)
    daily_ic_results[name] = {'series': ic_series, 'stats': ic_stats}
    
    sig = "***" if ic_stats['p_value'] < 0.01 else ("**" if ic_stats['p_value'] < 0.05 else ("*" if ic_stats['p_value'] < 0.10 else ""))
    print(f"{name:15s} | {ic_stats['ic_mean']:>+10.4f} | {ic_stats['ic_std']:>10.4f} | {ic_stats['ir']:>8.3f} | {ic_stats['t_stat']:>+8.2f} | {sig:>6s}")


In [ ]:
# 可视化IC时序
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (name, result) in enumerate(daily_ic_results.items()):
    ax = axes[idx]
    ic = result['series']
    stats = result['stats']
    
    # IC时序
    ic.plot(ax=ax, alpha=0.4, linewidth=0.5)
    ic.rolling(60).mean().plot(ax=ax, linewidth=2, label='60日MA')
    
    ax.axhline(0, color='black', linestyle='--', alpha=0.5)
    ax.axhline(stats['ic_mean'], color='red', linestyle='--', alpha=0.7, 
               label=f"Mean: {stats['ic_mean']:.4f}")
    
    ax.set_title(f"{name} 因子IC (IR={stats['ir']:.3f})", fontsize=12)
    ax.set_xlabel('')
    ax.legend(loc='upper left', fontsize=9)

# 隐藏多余的子图
if len(daily_ic_results) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.suptitle('日频因子IC时序分析', y=1.02, fontsize=14)
plt.show()


## 4. 多周期测试

比较因子在日频、周频、月频下的表现差异。


In [ ]:
# 多周期测试
def test_multi_frequency(close, volume, high, low, frequencies):
    """测试多个频率"""
    all_results = []
    
    for freq, freq_name, lookback in frequencies:
        print(f"\n📅 测试 {freq_name} 频率...")
        
        # 重采样数据
        if freq != 'D':
            close_f = close.resample(freq).last()
            volume_f = volume.resample(freq).sum()
            high_f = high.resample(freq).max()
            low_f = low.resample(freq).min()
        else:
            close_f, volume_f, high_f, low_f = close, volume, high, low
        
        returns_f = close_f.pct_change()
        
        # 构建因子
        factors = build_factors(returns_f, volume_f, high_f, low_f, close_f, lookback=lookback)
        
        # 测试每个因子
        for name, factor in factors.items():
            try:
                _, ic_stats = analyze_factor_ic(factor, returns_f, name)
                all_results.append({
                    'Factor': name,
                    'Frequency': freq_name,
                    'IC_Mean': ic_stats['ic_mean'],
                    'IC_Std': ic_stats['ic_std'],
                    'IR': ic_stats['ir'],
                    't_stat': ic_stats['t_stat'],
                    'p_value': ic_stats['p_value'],
                    'Significant': ic_stats['p_value'] < 0.05
                })
            except Exception as e:
                print(f"  ⚠️ {name}: {e}")
    
    return pd.DataFrame(all_results)

# 定义频率参数
frequencies = [
    ('D', 'Daily', 20),      # 日频, 20日回溯
    ('W', 'Weekly', 12),     # 周频, 12周回溯
    ('ME', 'Monthly', 6)      # 月频, 6月回溯
]

# 运行多周期测试
results_df = test_multi_frequency(close, volume, high, low, frequencies)

print("\n" + "="*80)
print("📊 多周期因子测试结果汇总")
print("="*80)


In [ ]:
# 结果可视化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. 各因子在不同频率下的IC Mean
pivot_ic = results_df.pivot(index='Factor', columns='Frequency', values='IC_Mean')
pivot_ic = pivot_ic[['Daily', 'Weekly', 'Monthly']]

ax1 = axes[0]
pivot_ic.plot(kind='bar', ax=ax1, width=0.8, edgecolor='black')
ax1.axhline(0, color='black', linestyle='--', alpha=0.5)
ax1.set_title('各因子在不同频率下的 IC Mean', fontsize=14)
ax1.set_xlabel('因子')
ax1.set_ylabel('IC Mean')
ax1.legend(title='频率')
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 2. IR热力图
pivot_ir = results_df.pivot(index='Factor', columns='Frequency', values='IR')
pivot_ir = pivot_ir[['Daily', 'Weekly', 'Monthly']]

ax2 = axes[1]
sns.heatmap(pivot_ir, annot=True, fmt='.3f', cmap='RdYlGn', center=0, 
            ax=ax2, linewidths=0.5)
ax2.set_title('因子 IR 热力图 (|IR| > 0.3 为优秀)', fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
# 显示详细结果表格
print("\n📋 完整测试结果:")
display(results_df.round(4).style.background_gradient(subset=['IC_Mean', 'IR'], cmap='RdYlGn', vmin=-0.1, vmax=0.1))

# 统计显著因子
significant = results_df[results_df['Significant']]
print(f"\n✅ 显著有效因子数量: {len(significant)} / {len(results_df)}")

if len(significant) > 0:
    print("\n🏆 显著因子列表:")
    for _, row in significant.iterrows():
        direction = "正向" if row['IC_Mean'] > 0 else "负向"
        print(f"   • {row['Factor']} ({row['Frequency']}): IC={row['IC_Mean']:+.4f}, "
              f"IR={row['IR']:.3f}, {direction}预测")
else:
    print("\n⚠️ 没有因子在 5% 水平下显著")
    print("   可能原因: 市场效率较高 / 样本期特殊 / 需要更复杂的因子")


## 5. 结论

### 关键发现

1. **多周期验证的重要性**: 同一因子在不同频率下表现可能差异显著
2. **因子稳定性**: 真正有效的因子应在多个周期下都保持预测能力
3. **市场效率**: 美股市场效率较高，简单因子的预测能力可能有限

### 后续步骤

1. 对有效因子进行 **Beta中性化** (阶段三)
2. 使用 **Fama-MacBeth回归** 进行最终假设检验 (阶段四)
3. 考虑交易成本后的实际收益

### 注意事项

- 以上仅为示例，实际研究需要更大的股票样本
- 需要考虑生存偏差、前视偏差等问题
- 因子有效性可能随时间变化
